## Map opiod death rate data (2016)
 Gets geojson data (json data with coordinates that form the shape of a congressional district) <br />
 Uses folium library to display map



In [1]:
# https://github.com/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb
# json data: http://eric.clst.org/tech/usgeojson/
# https://en.wikipedia.org/wiki/GeoJSON
# run pip install folium first

import os
import folium
import json
import pandas as pd
from branca.colormap import linear
import branca.colormap as cm
#print(folium.__version__)

In [2]:
us_cdistricts = os.path.join('data', 'gz_2010_us_500_11_5m.json')
geo_json_data = json.load(open(us_cdistricts))

### Json format:

{
    "type": "FeatureCollection",
    "features": [
        {   
            "type": "Feature", 
            "properties": {     
              <strong>  "GEO_ID": "5001100US1902", </strong>
                "STATE": "19", 
                "CD": "02", 
                "NAME": "2", 
                "LSAD": "CD", 
                "CENSUSAREA": 7563.446000 }, 
            "geometry": {     
                "type": "Polygon", 
                "coordinates": [ [ [ -93.097296, 40.584014 ], [ -93.098507, 40.583973 ], [ -93.135802, 40.582854 ], [ -93.260612, 40.580797 ], [ -93.317605, 40.580671 ], [ .... 

In [4]:
opioid_df = pd.read_csv('data/districtdata.csv')
oDeaths_df= opioid_df[(opioid_df['INDICATOR']=='drugdeathrate')& (opioid_df['YEAR']==2016)]
clnDeaths_df=oDeaths_df.fillna(0)
clnDeaths_df.reset_index(drop=True, inplace=True)

# prefix 0 to congressional district numbers <10 

stateValues=clnDeaths_df['STATEFP']
distValues=clnDeaths_df['DISTRICT']
#print(len(stateValues))
def makeGeoId(statecol,districtcol):
    geoIds=[]
    counter=0
    while counter<len(stateValues):
        geoId='5001100US'
        if int(statecol[counter]) <10:
            geoId+='0'
        geoId+=str(statecol[counter])
        if int(districtcol[counter])<10:
            geoId+='0'
        geoId+=str(districtcol[counter])
        counter+=1
        #print(statecol[counter],districtcol[counter],geoId)
        geoIds.append(geoId)
    return geoIds

geoIDs=makeGeoId(stateValues, distValues)
clnDeaths_df['GEO_ID']=geoIDs
clnDeaths_df.head()
#drop unused info, keep values and GeoID


,Unnamed: 0,STATEFP,DISTRICT,YEAR,INDICATOR,VALUE,STATE,STATEABBREVIATION,GEO_ID
0,79833,1,1,2016,drugdeathrate,12.7,Alabama,AL,5001100US0101
1,79847,1,2,2016,drugdeathrate,0.3,Alabama,AL,5001100US0102
2,79861,1,3,2016,drugdeathrate,6.8,Alabama,AL,5001100US0103
3,79875,1,4,2016,drugdeathrate,11.7,Alabama,AL,5001100US0104
4,79889,1,5,2016,drugdeathrate,11.5,Alabama,AL,5001100US0105


### Keep death rate value and corresponding geo location ID

In [7]:
clnDeathsf_df=clnDeaths_df[['GEO_ID','VALUE']]
# Added 'US0' for any json files not in opiod data frame
clnDeathsf_df.loc[-1] =['US0',0]
clnDeathsf_df.index = clnDeathsf_df.index + 1 
clnDeathsf_df.sort_values('GEO_ID')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,GEO_ID,VALUE
1,5001100US0101,12.7
2,5001100US0102,0.3
3,5001100US0103,6.8
4,5001100US0104,11.7
5,5001100US0105,11.5
6,5001100US0106,25.5
7,5001100US0107,18.4
8,5001100US0201,17.3
9,5001100US0401,19.7
10,5001100US0402,23.3


### Create a color map, where color corresponds to death rate value.
Returns RGB value 

In [8]:

#colormap = linear.YlGn_09.scale(clnDeaths_df.VALUE.min(),clnDeaths_df.VALUE.max()) #pretty but too light
colormap = cm.LinearColormap(colors=['blue','maroon'], index=[0,28], vmin=clnDeaths_df.VALUE.min(),vmax=clnDeaths_df.VALUE.max()/1.25)
print(colormap(12.7))

colormap

#3a008b


In [9]:
deathrate_dict = clnDeathsf_df.set_index('GEO_ID')['VALUE']

### Create map based on GEO_ID from json file
There is at least 1 but few GEOIDs in GeoJson file but not in opdiod dataframe, for those IDs use colormap whith value of 0.

In [11]:

m = folium.Map([43, -100], zoom_start=4)
folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {  
        'fillColor': colormap(deathrate_dict.get(feature['properties']['GEO_ID'], 0.0)),
        'color': 'gray',
        'weight': 1,
        'dashArray': '5, 5'
    }
).add_to(m)
# save as html 
m.save(os.path.join('data', 'GeoJSON_USDeathR.html'))
